In [ ]:
df<-read.csv('d:/data/customer/customer.csv')
head(df)
dim(df)

In [ ]:
(tbl<-table(df$Churn))
barplot(tbl,beside=T,legend=T,col=rainbow(2))

In [ ]:
library(ROSE)
df_samp<-ovun.sample(Churn ~. , data=df, seed=1, method='under',
                    N=483*2)$data
(tbl<-table(df_samp$Churn))

In [ ]:
library(caret)
set.seed(123)
idx_train <- createDataPartition(y=df$Churn, p=0.8, list=F)
train<-df[idx_train,]
X_train<-train[,-17]
y_train<-train[,17]
test<-df[-idx_train,]
X_test<-test[,-17]
y_test<-test[,17]

In [ ]:
library(reshape)
#scaling 전의 박스플롯
meltData<-melt(X_train)
boxplot(data=meltData,value~variable)

In [ ]:
#scale() 변수들의 평균,표준편차들 비슷하게 맞추고 데이터프레임으로 변환
X_train_scaled<-as.data.frame(scale(X_train))
X_test_scaled<-as.data.frame(scale(X_test))
#스케일링된 독립변수들과 종속변수 연결
train_scaled<-cbind(X_train_scaled,Churn=y_train)
test_scaled<-cbind(X_test_scaled,Churn=y_test)

In [ ]:
#scaling 후의 박스플롯
meltData<-melt(X_train_scaled)
boxplot(data=meltData,value~variable)

In [ ]:
#인공신경망 모형 생성
library(nnet)
model<-nnet(Churn ~ ., data=train_scaled, size=20)
summary(model)

In [ ]:
#인공신경망 그래프
library(devtools)

source_url('https://gist.githubusercontent.com/Peque/41a9e20d6687f2f3108d/raw/85e14f3a292e126f1454864427e3a189c2fe33f3/nnet_plot_update.r')

plot.nnet(model)

In [ ]:
#로지스틱 회귀분석 모형
model<-glm(Churn ~ . , data=train_scaled, family=binomial)
summary(model)

In [ ]:
#후진제거법
reduced<-step(model,direction='backward')

In [ ]:
library(NeuralNetTools)
garson(model)#변수 중요도 그래프
#변수의 중요도 : int.l.plan > custServ.Calls> Day.Charge

In [ ]:
#학습용 데이터셋의 예측 정확도
pred<-predict(model,X_train_scaled)
result<-round(pred)
table(y_train,result)
mean(y_train == result)

In [ ]:
#검증용 데이터셋의 예측 정확도
pred<-predict(model,X_test_scaled)
result<-round(pred)
table(y_test,result)
mean(y_test == result)